In [2]:
import pandas as pd
import pathlib

In [3]:
path = pathlib.Path("/home/ubuntu/giovanni/data")

df = pd.read_parquet(path / 'profiles_var_mad_int.parquet')

df.head()



,Metadata_Source,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Cells_AreaShape_Area,Cells_AreaShape_BoundingBoxArea,Cells_AreaShape_BoundingBoxMaximum_X,Cells_AreaShape_BoundingBoxMaximum_Y,Cells_AreaShape_BoundingBoxMinimum_X,Cells_AreaShape_BoundingBoxMinimum_Y,...,Nuclei_Texture_Variance_RNA_10_02_256,Nuclei_Texture_Variance_RNA_10_03_256,Nuclei_Texture_Variance_RNA_3_00_256,Nuclei_Texture_Variance_RNA_3_01_256,Nuclei_Texture_Variance_RNA_3_02_256,Nuclei_Texture_Variance_RNA_3_03_256,Nuclei_Texture_Variance_RNA_5_00_256,Nuclei_Texture_Variance_RNA_5_01_256,Nuclei_Texture_Variance_RNA_5_02_256,Nuclei_Texture_Variance_RNA_5_03_256
0,source_2,1053597806,A01,JCP2022_085227,-0.988041,-0.907001,0.096069,0.323007,0.146602,0.534867,...,1.605010,1.693312,1.615753,1.604152,1.624943,1.612151,1.589302,1.594195,1.627130,1.610461
1,source_2,1053597806,K22,JCP2022_049123,-0.247098,-0.389633,-0.828097,0.727857,-0.708414,0.887224,...,-0.696735,-0.547206,-0.737023,-0.739017,-0.741798,-0.741577,-0.724849,-0.714124,-0.755699,-0.729048
2,source_2,1053597806,K21,JCP2022_025146,0.882814,0.635229,-0.866758,1.593473,-0.798442,1.682966,...,0.427341,0.699385,0.506531,0.478770,0.518382,0.476467,0.454404,0.473200,0.500973,0.484033
3,source_2,1053597806,K20,JCP2022_109006,1.551945,1.409719,-0.624934,-1.552434,-0.634504,-1.846982,...,-0.016852,0.321772,0.019403,-0.023218,0.025426,-0.020011,-0.036913,-0.064012,-0.037481,-0.048811
4,source_2,1053597806,K19,JCP2022_040739,0.782228,0.394115,0.016284,0.335031,0.009199,0.228013,...,0.549137,0.589808,0.643868,0.616816,0.655204,0.615139,0.594326,0.588221,0.632808,0.615094


In [33]:
df.Metadata_Source.unique()

<StringArray>
[ 'source_2',  'source_6',  'source_8',  'source_5',  'source_3',  'source_7',
 'source_11', 'source_10',  'source_9',  'source_1']
Length: 10, dtype: string

In [11]:
import polars as pl
from broad_babel.query import get_mapper

INDEX_FILE = "https://raw.githubusercontent.com/jump-cellpainting/datasets/50cd2ab93749ccbdb0919d3adf9277c14b6343dd/manifests/profile_index.csv"
CRISPR_URL = pl.read_csv(INDEX_FILE).filter(pl.col("subset") == "compound_interpretable").item(0, "url")
profiles = pl.scan_parquet(CRISPR_URL)
print(profiles.collect_schema().names()[:6])

['Metadata_Source', 'Metadata_Plate', 'Metadata_Well', 'Metadata_JCP2022', 'Cells_AreaShape_Area', 'Cells_AreaShape_BoundingBoxArea']


In [14]:
jcp_ids = (
    profiles.select(pl.col("Metadata_JCP2022")).unique().collect().to_series().sort()
)
subsample = jcp_ids.sample(10, seed=42)
subsample = (*subsample, "JCP2022_800002")

In [20]:
name_mapper = get_mapper(
    (*subsample, "JCP2022_800002"),
    input_column="JCP2022",
    output_columns="JCP2022,standard_key",
)

In [17]:
name_mapper

{'JCP2022_028278': 'GWTBXQQPVRNIPZ-UHFFFAOYSA-N',
 'JCP2022_027032': 'GPXONAQDNOJZDS-UHFFFAOYSA-N',
 'JCP2022_062945': 'OCQUGGYNYZAZCD-UHFFFAOYSA-N',
 'JCP2022_034511': 'IEGWXHZAERRPAA-UHFFFAOYSA-N',
 'JCP2022_011184': 'CHOCUHAMWSTXQB-UHFFFAOYSA-N',
 'JCP2022_035521': 'IJXNECHDDWQPST-UHFFFAOYSA-N',
 'JCP2022_037172': 'ISRUTBYDVYWAGL-UHFFFAOYSA-N',
 'JCP2022_001698': 'AIYDZKWHEKBPBD-UHFFFAOYSA-N',
 'JCP2022_017646': 'DQUXSWDRCMFRKS-UHFFFAOYSA-N',
 'JCP2022_018617': 'DWHZCHWSVPXJNC-UHFFFAOYSA-N',
 'JCP2022_800002': 'non-targeting'}

In [22]:
import anndata as ad
adata = ad.read_h5ad(path / '20250417.diff_vision_scores_pseudobulk.public.h5ad')

In [24]:
adata.obs

,plate,Cell_Name_Vevo,Cell_ID_Cellosaur,drug,concentration,concentration_unit
0,4,A549,CVCL_0023,8-Hydroxyquinoline,0.05,uM
1,4,HS-578T,CVCL_0332,8-Hydroxyquinoline,0.05,uM
2,4,HCT15,CVCL_0292,8-Hydroxyquinoline,0.05,uM
3,4,HOP62,CVCL_1285,8-Hydroxyquinoline,0.05,uM
4,4,SK-MEL-2,CVCL_0069,8-Hydroxyquinoline,0.05,uM
...,...,...,...,...,...,...
60771,12,hTERT-HPNE,CVCL_C466,Adagrasib,0.05,uM
60772,12,NCI-H23,CVCL_1547,Adagrasib,0.05,uM
60773,12,NCI-H1792,CVCL_1495,Adagrasib,0.05,uM
60774,12,NCI-H2030,CVCL_1517,Adagrasib,0.05,uM


In [27]:
from datasets import load_dataset
drug_metadata =  load_dataset("tahoebio/Tahoe-100M","drug_metadata", split="train")


Resolving data files:   0%|          | 0/3388 [00:00<?, ?it/s]

In [29]:
drug_metadata.to_pandas()

,drug,targets,moa-broad,moa-fine,human-approved,clinical-trials,gpt-notes-approval,canonical_smiles,pubchem_cid
0,Talc,None,unclear,unclear,yes,yes,Talc used in pharma and cosmetics; safety unde...,[OH-].[OH-].[O-][Si]12O[Si]3(O[Si](O1)(O[Si](O...,165411828.0
1,Bortezomib,PSMB5,inhibitor/antagonist,Proteasome inhibitor,yes,yes,Approved for multiple myeloma and mantle cell ...,B(C(CC(C)C)NC(=O)C(CC1=CC=CC=C1)NC(=O)C2=NC=CN...,387447.0
2,Ixazomib,PSMB5,inhibitor/antagonist,Proteasome inhibitor,yes,yes,Approved for multiple myeloma treatment.,B(C(CC(C)C)NC(=O)CNC(=O)C1=C(C=CC(=C1)Cl)Cl)(O)O,25183872.0
3,Ixazomib citrate,"PSMB1, PSMB2, PSMB5",inhibitor/antagonist,Proteasome inhibitor,yes,yes,Approved for multiple myeloma treatment as par...,B1(OC(=O)C(O1)(CC(=O)O)CC(=O)O)C(CC(C)C)NC(=O)...,56844015.0
4,Lactate (calcium),None,unclear,unclear,yes,yes,"Used in medical settings, but not specifically...",C.CC(C(=O)[O-])O.[Ca+2],168311648.0
...,...,...,...,...,...,...,...,...,...
374,Verteporfin,YAP1,inhibitor/antagonist,unclear,yes,yes,Used in photodynamic therapy for macular degen...,None,NaN
375,Quinidine (15% dihydroquinidine),KCNH2,inhibitor/antagonist,unclear,yes,yes,Approved for arrhythmias as part of quinine al...,COC1=CC2=C(C=CN=C2C=C1)[C@@H]([C@H]3C[C@@H]4CC...,441074.0
376,Canagliflozin (hemihydrate),SLC5A2,inhibitor/antagonist,Glucose transporter inhibitor,yes,yes,Approved for type 2 diabetes.,CC1=C(C=C(C=C1)[C@H]2[C@@H]([C@H]([C@@H]([C@H]...,24997615.0
377,Osimertinib (mesylate),EGFR,inhibitor/antagonist,EGFR/ERBB inhibitor,yes,yes,Approved for non-small cell lung cancer treatm...,CN1C=C(C2=CC=CC=C21)C3=NC(=NC=C3)NC4=C(C=C(C(=...,78357807.0


In [30]:
mapper = get_mapper(query="compound", input_column="plate_type", output_columns="JCP2022,pert_type")


In [31]:
mapper

{'JCP2022_014696': 'trt',
 'JCP2022_040501': 'trt',
 'JCP2022_051313': 'trt',
 'JCP2022_061545': 'trt',
 'JCP2022_062817': 'trt',
 'JCP2022_043906': 'trt',
 'JCP2022_027359': 'trt',
 'JCP2022_026224': 'trt',
 'JCP2022_052479': 'trt',
 'JCP2022_071231': 'trt',
 'JCP2022_072172': 'trt',
 'JCP2022_103039': 'trt',
 'JCP2022_090879': 'trt',
 'JCP2022_080043': 'trt',
 'JCP2022_070637': 'trt',
 'JCP2022_102613': 'trt',
 'JCP2022_008834': 'trt',
 'JCP2022_095349': 'trt',
 'JCP2022_002648': 'trt',
 'JCP2022_035724': 'trt',
 'JCP2022_081339': 'trt',
 'JCP2022_022059': 'trt',
 'JCP2022_025846': 'trt',
 'JCP2022_018782': 'trt',
 'JCP2022_026446': 'trt',
 'JCP2022_063737': 'trt',
 'JCP2022_041599': 'trt',
 'JCP2022_043335': 'trt',
 'JCP2022_000492': 'trt',
 'JCP2022_079186': 'trt',
 'JCP2022_088316': 'trt',
 'JCP2022_011773': 'trt',
 'JCP2022_092722': 'trt',
 'JCP2022_041977': 'trt',
 'JCP2022_047220': 'trt',
 'JCP2022_047933': 'trt',
 'JCP2022_065216': 'trt',
 'JCP2022_099000': 'trt',
 'JCP2022_10

In [34]:
df = pd.read_parquet(path / 'compound_interpretable_features.parquet?download=1')
df

,Compartment,Feature,Channel,Suffix,Feature significance,Perturbation,Perturbation example image,Median,Gene Rank,Feature Rank,JCP2022,Resources,Synonyms
0,Cells,RadialDistributionFracAtD,,_mito_tubeness_3of16,0.08213,AIRPKJLLJCUQSV-UHFFFAOYSA-N,"{""img_src"": ""https://phenaid.ardigen.com/stati...",-0.964,999999,9,JCP2022_001647,None,None
1,Nuclei,AreaShapeZernike,,_6_2,0.00003,DTCQXSFBETURSZ-UHFFFAOYSA-N,"{""img_src"": ""https://phenaid.ardigen.com/stati...",1.244,999999,0,JCP2022_018053,None,None
2,Nuclei,RadialDistribution_RadialCV,Mito,_3of4,0.02041,UQEZJEIWCQNVQU-UHFFFAOYSA-N,"{""img_src"": ""https://phenaid.ardigen.com/stati...",-1.514,999999,4,JCP2022_090828,None,None
3,Cytoplasm,RadialDistribution_FracAtD,ER,_2of4,0.00227,MOQWDHSSODJEDU-UHFFFAOYSA-N,"{""img_src"": ""https://phenaid.ardigen.com/stati...",1.486,999999,2,JCP2022_055532,None,None
4,Cytoplasm,RadialDistribution_FracAtD,DNA,_1of4,0.01685,WPCDYOHCABUJAF-UHFFFAOYSA-N,"{""img_src"": ""https://phenaid.ardigen.com/stati...",1.097,999999,1,JCP2022_100243,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189393,Cells,Texture_SumEntropy,RNA,_10_01_256,0.02717,KHEBJWNXOZKKBY-UHFFFAOYSA-N,"{""img_src"": ""https://phenaid.ardigen.com/stati...",1.227,999999,2,JCP2022_044597,None,None
1189394,Nuclei,AreaShapeCompactness,,,0.00000,QGGLNUGOWVBAJC-UHFFFAOYSA-N,"{""img_src"": ""https://phenaid.ardigen.com/stati...",-1.906,999999,1,JCP2022_073238,None,None
1189395,Cells,AreaShapeMaxFeretDiameter,,,0.00138,SBJVCDHWJCQQIQ-UHFFFAOYSA-N,"{""img_src"": ""https://phenaid.ardigen.com/stati...",-1.688,999999,8,JCP2022_082092,None,None
1189396,Nuclei,Texture_Correlation,ER,_10_02_256,0.19502,FCQDLIHIOSXHMH-UHFFFAOYSA-N,"{""img_src"": ""https://phenaid.ardigen.com/stati...",1.206,999999,5,JCP2022_019780,None,None


In [35]:
df

,Compartment,Feature,Channel,Suffix,Feature significance,Perturbation,Perturbation example image,Median,Gene Rank,Feature Rank,JCP2022,Resources,Synonyms
0,Cells,RadialDistributionFracAtD,,_mito_tubeness_3of16,0.08213,AIRPKJLLJCUQSV-UHFFFAOYSA-N,"{""img_src"": ""https://phenaid.ardigen.com/stati...",-0.964,999999,9,JCP2022_001647,None,None
1,Nuclei,AreaShapeZernike,,_6_2,0.00003,DTCQXSFBETURSZ-UHFFFAOYSA-N,"{""img_src"": ""https://phenaid.ardigen.com/stati...",1.244,999999,0,JCP2022_018053,None,None
2,Nuclei,RadialDistribution_RadialCV,Mito,_3of4,0.02041,UQEZJEIWCQNVQU-UHFFFAOYSA-N,"{""img_src"": ""https://phenaid.ardigen.com/stati...",-1.514,999999,4,JCP2022_090828,None,None
3,Cytoplasm,RadialDistribution_FracAtD,ER,_2of4,0.00227,MOQWDHSSODJEDU-UHFFFAOYSA-N,"{""img_src"": ""https://phenaid.ardigen.com/stati...",1.486,999999,2,JCP2022_055532,None,None
4,Cytoplasm,RadialDistribution_FracAtD,DNA,_1of4,0.01685,WPCDYOHCABUJAF-UHFFFAOYSA-N,"{""img_src"": ""https://phenaid.ardigen.com/stati...",1.097,999999,1,JCP2022_100243,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189393,Cells,Texture_SumEntropy,RNA,_10_01_256,0.02717,KHEBJWNXOZKKBY-UHFFFAOYSA-N,"{""img_src"": ""https://phenaid.ardigen.com/stati...",1.227,999999,2,JCP2022_044597,None,None
1189394,Nuclei,AreaShapeCompactness,,,0.00000,QGGLNUGOWVBAJC-UHFFFAOYSA-N,"{""img_src"": ""https://phenaid.ardigen.com/stati...",-1.906,999999,1,JCP2022_073238,None,None
1189395,Cells,AreaShapeMaxFeretDiameter,,,0.00138,SBJVCDHWJCQQIQ-UHFFFAOYSA-N,"{""img_src"": ""https://phenaid.ardigen.com/stati...",-1.688,999999,8,JCP2022_082092,None,None
1189396,Nuclei,Texture_Correlation,ER,_10_02_256,0.19502,FCQDLIHIOSXHMH-UHFFFAOYSA-N,"{""img_src"": ""https://phenaid.ardigen.com/stati...",1.206,999999,5,JCP2022_019780,None,None


In [39]:
import pubchempy as pcp
import time

def get_compound_name_and_synonyms_from_inchikey(inchikey):
    """
    Retrieve a compound name and all possible synonyms based on an InChIKey using PubChemPy.
    
    Args:
        inchikey (str): The InChIKey of the compound
        
    Returns:
        dict: A dictionary containing the compound name, CID, and synonyms
    """
    result = {
        "inchikey": inchikey,
        "cid": None,
        "name": None,
        "synonyms": [],
        "status": "success"
    }
    
    try:
        # Step 1: Get compounds by InChIKey
        compounds = pcp.get_compounds(inchikey, 'inchikey')
    except Exception as e:
        result["status"] = "error"
        result["error"] = str(e)
        return result
    
    # if compounds:
        # compound = compounds[0]
    return compounds

# Example usage
inchikey = "ASMXXROZKSBQIH"
result = get_compound_name_and_synonyms_from_inchikey(inchikey)
print(result)


[Compound(76974213), Compound(11467167), Compound(11434515), Compound(11228026)]


In [41]:
for r in result:
    print(r.synonyms)

[]
['SCHEMBL72140', 'SCHEMBL12964939', 'SCHEMBL14021458', 'NS00004667']
['Aclidinium', 'Aclidinium ion', 'Aclidinium cation', 'UNII-K17VY42F6C', 'K17VY42F6C', '727649-81-2', 'CHEBI:65346', 'DTXSID00223070', '(3R)-3-[2-hydroxy(di-2-thienyl)acetoxy]-1-(3-phenoxypropyl)-1-azoniabicyclo[2.2.2]octane', '(3R)-3-(2-HYDROXY(DI-2-THIENYL)ACETOXY)-1-(3-PHENOXYPROPYL)-1-AZONIABICYCLO(2.2.2)OCTANE', '1-AZONIABICYCLO(2.2.2)OCTANE, 3-((HYDROXYDI-2-THIENYLACETYL)OXY)-1-(3-PHENOXYPROPYL)-, (3R)-', 'Aclidinio', 'DTXCID00145561', 'CHEMBL551466', 'ACLIDINIUM [VANDF]', 'ACLIDINIUM [WHO-DD]', 'SCHEMBL72141', 'GTPL7449', 'CHEMBL1194325', 'SCHEMBL15096073', 'BDBM50296331', 'DB08897', 'NCGC00386257-02', 'HY-159686', 'AB01565828_02', 'Q521173', 'BRD-K74363950-004-01-0', '[(3R)-1-(3-phenoxypropyl)-1-azoniabicyclo[2.2.2]octan-3-yl] 2-hydroxy-2,2-dithiophen-2-ylacetate', '[(8R)-1-[3-(phenoxy)propyl]-1-azoniabicyclo[2.2.2]octan-8-yl] 2-hydroxy-2,2-di(thiophen-2-yl)acetate']
['C6V6LNU2YL', 'SCHEMBL2075481', 'SCHEMB